In [ ]:
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama


# Configure Ollama LLM
ollama_llm = OllamaLLM(
    #model="llama3.2:latest",
    model="mistral:7b",
    base_url="http://localhost:11434",
    temperature=0.1
)

chat_llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0,
    # other params...
)

In [83]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = chat_llm.bind_tools(tools)

In [85]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '82d145f2-1b89-4983-bbdb-96df7163fb86', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'bf0f7b39-b136-4f4e-85fc-7fe1fcaf34c5', 'type': 'tool_call'}]


In [86]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-27T07:20:53.06408Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1551636700, 'load_duration': 1108059200, 'prompt_eval_count': 231, 'prompt_eval_duration': 178000000, 'eval_count': 44, 'eval_duration': 262000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-412f7cb0-4665-4209-bdda-688360b57644-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '82d145f2-1b89-4983-bbdb-96df7163fb86', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'bf0f7b39-b136-4f4e-85fc-7fe1fcaf34c5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 231, 'output_tokens': 44, 'total_tokens': 275}),
 ToolMessage(content='36', name='multiply', tool_call_id='82d145f2-1b89-4983-bbdb-96df7163fb86'),
 T

In [89]:
result = llm_with_tools.invoke(messages)



In [92]:
result

AIMessage(content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-27T07:21:39.5119199Z', 'done': True, 'done_reason': 'stop', 'total_duration': 450985300, 'load_duration': 19644400, 'prompt_eval_count': 125, 'prompt_eval_duration': 3000000, 'eval_count': 25, 'eval_duration': 426000000, 'message': Message(role='assistant', content='The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.', images=None, tool_calls=None)}, id='run-79efc280-6d42-4b73-9ff6-501f6e000614-0', usage_metadata={'input_tokens': 125, 'output_tokens': 25, 'total_tokens': 150})

In [91]:
result.content

'The result of 3 * 12 is 36.\n\nThe result of 11 + 49 is 60.'